In [1]:
from django.core.files import File
img = ImageFile()
with open('./images/02-NYH-underviser-AN-1.jpg', 'rb') as fp:
    imf=File(fp)
    img.source_file.save('foobar.jpg', imf)


In [2]:
from apps.photo.cropping.crop_engine import CloseCropEngine

In [7]:
from wand.image import Image

In [41]:
from sorl.thumbnail.engines.wand_engine import Engine, EngineBase

In [3]:
from apps.photo.tasks import post_save_task

In [4]:
def get_image(im_file):
    with im_file.source_file as fp:
        return Image(blob=fp.read())

#image_file = ImageFile.objects.order_by('?').first()
image_file = ImageFile.objects.get(pk=41050)
print(image_file.pk, image_file, image_file.size, image_file.full_height, image_file.full_width)

41050 07-OYE-debatt-AN-1.jpg 195513 1000 1500


In [5]:
from sorl.thumbnail import get_thumbnail
from IPython.display import HTML
print(image_file.get_cropping_method_display())
image_file.cropping_method = ImageFile.CROP_PENDING
image_file.save()
post_save_task(image_file)
post_save_task(image_file)

pending


13:46:12 2017-04-24 [ INFO] apps.photo.tasks     tasks.py:53   (post_save_task)
	autocrop 41050 07-OYE-debatt-AN-1.jpg detaljer

13:46:12 2017-04-24 [ INFO] apps.photo.tasks     tasks.py:59   (post_save_task)
	clear thumbs for 41050 07-OYE-debatt-AN-1.jpg



In [6]:
im1 = get_thumbnail(image_file.source_file, '400x300', crop_box=image_file.get_crop_box(), expand=-0.7)
im2 = get_thumbnail(image_file.source_file, '400x300', crop_box=image_file.get_crop_box(), expand=-0.2)
im3 = get_thumbnail(image_file.source_file, '400x300', crop_box=image_file.get_crop_box(), expand=0)
im4 = get_thumbnail(image_file.source_file, '400x300', crop_box=image_file.get_crop_box(), expand=0.6)
im5 = get_thumbnail(image_file.source_file, '400x300', crop_box=image_file.get_crop_box(), expand=1.0)

HTML(f"""
<img src={im1.url} /> 
<img src={im2.url} />
<img src={im3.url} /> 
<img src={im4.url} />
<img src={im5.url} /> 
""")

In [7]:
im1 = get_thumbnail(image_file.source_file, '550x200', crop_box=image_file.get_crop_box(), expand=1)
im2 = get_thumbnail(image_file.source_file, '550x200', crop_box=image_file.get_crop_box(), expand=-0.3)
im3 = get_thumbnail(image_file.source_file, '550x200', crop_box=image_file.get_crop_box(), expand=-0.7)
im4 = get_thumbnail(image_file.source_file, '550x550', crop_box=image_file.get_crop_box(), expand=-0.5)
im5 = get_thumbnail(image_file.source_file, '100x550', crop_box=image_file.get_crop_box(), expand=0)
im6 = get_thumbnail(image_file.source_file, '100x550', crop_box=image_file.get_crop_box(), expand=-0.9)
print(image_file, image_file.crop_box)
HTML(f"""
<img src={im1.url} /> 
<img src={im2.url} />
<img src={im3.url} /> 
<img src={im4.url} />
<img src={im5.url} />
<img src={im6.url} />
""")

07-OYE-debatt-AN-1.jpg {"left": 0.1841, "top": 0.2071, "bottom": 0.8834, "right": 0.8816, "x": 0.4639, "y": 0.4673}


In [ ]:
image_file.pk